In [1]:
from Libor_Formula.TaskC_FormulaNewPoints import evaluate

In [2]:
import os
import sys
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import copy
import numpy as np
import pickle as pkl
from pyhms.hms import hms
from pyhms.tree import DemeTree
from pyhms.config import EALevelConfig, CMALevelConfig
from pyhms.demes.single_pop_eas.sea import SEA
from pyhms.problem import EvalCutoffProblem, FunctionProblem
from pyhms.sprout.sprout_mechanisms import SproutMechanism, get_simple_sprout
from pyhms.sprout.sprout_filters import NBC_FarEnough, DemeLimit, LevelLimit
from pyhms.sprout.sprout_generators import NBC_Generator
from pyhms.stop_conditions.gsc import singular_problem_eval_limit_reached
from pyhms.stop_conditions.usc import metaepoch_limit, dont_stop

In [4]:
import pyade.ilshade
from cma import fmin2, CMAEvolutionStrategy

----------

In [5]:
def check_constraints(obtained_parameters):
    w = obtained_parameters
    if w[5] + w[6] > 0 and w[3] * w[4] - w[5] > 0 and w[3] * w[4] - w[5] - w[6] > 0 and abs(0.0322 * (w[5] + w[6]) - w[0]) - abs(w[0]) > 0:
        return True
    else:
        return False

In [6]:
def constrained_evaluate(w):
    if check_constraints(w):
        return evaluate(w)
    else:
        return 2*evaluate(w) + 0.1

In [7]:
def get_historic_best(tree: DemeTree) -> float:
    minimum = 2147483647.0

    for _, deme in tree.all_demes:
        for pop in deme.history:
            for ind in pop:
                if ind.fitness < minimum and check_constraints(ind.genome):
                    minimum = ind.fitness
    return minimum

In [8]:
OLD_CONFIG = {'far_enough': 16.85391591224264, 'generations1': 2, 'generations2': 6, 'level_limit': 4, 'meataepoch2': 6, 'mutation1': 2.5459184638416805, 'mutation2': 0.2906046628084067, 'pop1': 78, 'pop2': 35, 'sample_dev2': 0.9618745655809818}
NEW_CONFIG = {'generations1': 1, 'generations2': 2, 'level_limit': 5, 'meataepoch2': 6, 'mutation1': 0.4789120293659853, 'mutation2': 0.16658384975799623, 'nbc_cut': 1.7549302039405932, 'nbc_far': 3.9921360310330574, 'nbc_trunc': 0.29400968309797937, 'pop1': 103, 'pop2': 24, 'sample_dev2': 0.9038384697854606}
BOUNDS = np.array([[0, 500], [0, 500], [0, 1000], [0, 2500], [0, 2500], [-250, 250], [-250, 250], [0, 1000]])
PARAMETER_MULTIPLIER = np.mean([b[1] - b[0] for b in BOUNDS]) / 10.0 # Divided by BBOB domain size per dimension
DIM_NUMBER = 8
LIBOR_TASK = EvalCutoffProblem(FunctionProblem(constrained_evaluate, maximize=False), 9500, 0.0)

In [ ]:
def new_evaluator_sea_2(config, eval_limit, dim_number, function_problem, random_state):
    gsc = singular_problem_eval_limit_reached(limit=eval_limit)
    sprout = SproutMechanism(NBC_Generator(config["nbc_cut"], config["nbc_trunc"]), 
                                [NBC_FarEnough(config["nbc_far"], 2), DemeLimit(1)], 
                                [LevelLimit(config["level_limit"])])
    problem=copy.deepcopy(function_problem)
    options = {'hibernation': False, 'random_state': random_state}

    config_sea2 = [
    EALevelConfig(
        ea_class=SEA, 
        generations=config["generations1"],
        problem=problem, 
        bounds=BOUNDS, 
        pop_size=config["pop1"],
        mutation_std=config["mutation1"]*PARAMETER_MULTIPLIER,
        lsc=dont_stop()
        ),
    EALevelConfig(
        ea_class=SEA, 
        generations=config["generations2"], 
        problem=problem, 
        bounds=BOUNDS, 
        pop_size=config["pop2"],
        mutation_std=config["mutation2"]*PARAMETER_MULTIPLIER,
        sample_std_dev=config["sample_dev2"],
        lsc=metaepoch_limit(config["meataepoch2"]),
        run_minimize=True
        )
    ]
    tree = hms(level_config=config_sea2, gsc=gsc, sprout_cond=sprout, options=options)
    return tree

def old_evaluator_sea_2(config, eval_limit, dim_number, function_problem, random_state):
    gsc = singular_problem_eval_limit_reached(limit=eval_limit)
    sprout = get_simple_sprout(far_enough=config['far_enough']*PARAMETER_MULTIPLIER, level_limit=config['level_limit'])
    problem=copy.deepcopy(function_problem)
    options = {'hibernation': False, 'random_state': random_state}

    config_sea2 = [
    EALevelConfig(
        ea_class=SEA, 
        generations=config["generations1"],
        problem=problem, 
        bounds=BOUNDS, 
        pop_size=config["pop1"],
        mutation_std=config["mutation1"]*PARAMETER_MULTIPLIER,
        lsc=dont_stop()
        ),
    EALevelConfig(
        ea_class=SEA, 
        generations=config["generations2"], 
        problem=problem, 
        bounds=BOUNDS, 
        pop_size=config["pop2"],
        mutation_std=config["mutation2"]*PARAMETER_MULTIPLIER,
        sample_std_dev=config["sample_dev2"],
        lsc=metaepoch_limit(config["meataepoch2"]),
        run_minimize=True
        )
    ]
    tree = hms(level_config=config_sea2, gsc=gsc, sprout_cond=sprout, options=options)
    return tree

In [ ]:
old_bests = []
for i in range(50):
    if i % 10 == 0:
        print("gen {gen}".format(gen=i))
    old_tree = old_evaluator_sea_2(OLD_CONFIG, 9500, DIM_NUMBER, LIBOR_TASK, random_state=i+1)
    old_tree.run()
    old_bests.append(get_historic_best(old_tree))

In [ ]:
print("{:.1e}".format(np.mean(old_bests)), "{:.1e}".format(np.std(old_bests)))

In [ ]:
new_bests = []
for i in range(50):
    if i % 10 == 0:
        print("gen {gen}".format(gen=i))
    new_tree = new_evaluator_sea_2(NEW_CONFIG, 9500, DIM_NUMBER, LIBOR_TASK, random_state=i+1)
    new_tree.run()
    new_bests.append(get_historic_best(new_tree))

In [ ]:
print("{:.1e}".format(np.mean(new_bests)), "{:.1e}".format(np.std(new_bests)))

------------

In [9]:
def create_ilshade_callback(pop, fit):
    def ilshade_callback(**kwargs):
        pop.append(kwargs['population'])
        fit.append(kwargs['fitness'])
    return ilshade_callback

def create_cma_callback(pop):
    def cma_callback(current_cma: CMAEvolutionStrategy):
        pop.append(current_cma.pop_sorted)
    return cma_callback
    
def get_best_under_constraints(pop, fit):
    minima = 2147483647.0
    for i in range(len(pop)):
        if fit[i] < minima and check_constraints(pop[i]):
            minima = fit[i]
    return minima

In [10]:
algorithm = pyade.ilshade

ilshade_pop = []
ilshade_fit = []

params = algorithm.get_default_params(dim=8)
params['bounds'] = BOUNDS
params['func'] = constrained_evaluate
params['max_evals'] = 9500
params['callback'] = create_ilshade_callback(ilshade_pop, ilshade_fit)
params['population_size'] = 79
params['memory_size']

ilshade_bests = []
for i in range(50):
    if i % 10 == 0:
        print("gen {gen}".format(gen=i))
    random.seed(i+1)
    np.random.seed(i+1)
    solution, fitness = algorithm.apply(**params)
    ilshade_bests.append(get_best_under_constraints([ind for pop in ilshade_pop for ind in pop], [fit for fpop in ilshade_fit for fit in fpop]))
    ilshade_pop.clear()
    ilshade_fit.clear()

gen 0
gen 10
gen 20
gen 30
gen 40


In [11]:
print("{:.1e}".format(np.mean(ilshade_bests)), "{:.1e}".format(np.std(ilshade_bests)))

3.5e-04 2.1e-04


In [12]:
cma_lb = [b[0] for b in BOUNDS]
cma_ub = [b[1] for b in BOUNDS]
start = np.mean(np.stack([cma_lb, cma_ub]), axis=0)
sigma0 = 3.198595880444435*PARAMETER_MULTIPLIER
cma_pops = []
cma_call = create_cma_callback(cma_pops)

options = {'bounds': [cma_lb, cma_ub], 'verbose': -9, 'maxfevals': 9500, 'seed':i+1, 'randn':np.random.randn, 'popsize': 15, 'tolfun': 3.262297273400133e-10}

cma_bests = []
for i in range(50):
    if i % 10 == 0:
        print("gen {gen}".format(gen=i))
    random.seed(i+1)
    np.random.seed(i+1)
    best, es = fmin2(constrained_evaluate, start, sigma0, options=options, bipop=True, restart_from_best=True, restarts=10, callback=cma_call, incpopsize=2)
    whole_pop = [(pop, evaluate(pop)) for popul in cma_pops for pop in popul]
    cma_bests.append(get_best_under_constraints([ind[0] for ind in whole_pop], [ind[1] for ind in whole_pop]))
    cma_pops.clear()

gen 0
gen 10
gen 20
gen 30
gen 40


In [13]:
print("{:.1e}".format(np.mean(cma_bests)), "{:.1e}".format(np.std(cma_bests)))

4.0e-03 8.7e-19
